In [1]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

creds = None
with open('../../secret.json') as creds_file:
    creds = json.load(creds_file)


# Set up the Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

print('hi')


hi


In [2]:
# import time
# 
# artist_name = "Linkin Park"
# song_name = "Breaking the Habit"
# 
# try:
#     result = sp.search(q=f'track:{song_name} artist:{artist_name}', type='track')
#     print(result)
# except spotipy.SpotifyException as e:
#     print(f"An error occurred: {e}")
# finally:
#     # Introduce a delay between requests (e.g., 1 second)
#     time.sleep(1)


In [3]:
import time

def get_song_release_year(song_name, artist_name):
    time.sleep(1)
    # Search for the song
    results = sp.search(q=f'track:{song_name} artist:{artist_name}', type='track', limit=1)

    # Extract the release year from the first search result
    if len(results['tracks']['items']) > 0:
        release_date = results['tracks']['items'][0]['album']['release_date']
        release_year = release_date.split('-')[0]
        return release_year

    # Return None if no results were found
    return None

In [ ]:
artist_name = "Linkin Park"
song_name = "Breaking the Habit"
try:
    print(get_song_release_year(song_name, artist_name))
except Exception as e:
    print(str(e))



In [ ]:
import pandas as pd
from time import sleep
from tqdm import tqdm

cddb_df = pd.read_csv('../../cleaning_stages/3_category_genre/cddb_2.tsv', sep='\t')
tracks_df = pd.read_csv('../../cleaning_stages/3_category_genre/cddb_tracks.tsv', sep='\t')

cddb_df = cddb_df[cddb_df['year'].isnull()]
cddb_df = cddb_df[['artist', 'tracks', 'year']]

print(f'num null years: ${len(cddb_df)}')

def start_repair():
    num_repairs = 0
    num_not_repairable = 0
    
    total_rows = len(cddb_df)
    for _, cddb_row in tqdm(cddb_df.iterrows(), total=total_rows, desc='Processing rows', ncols=100):
        
        artist_name = cddb_row['artist']

        # Grab the first song of the track
        tracks_for_album = tracks_df[tracks_df['Album Id'] == cddb_row['tracks']]
        
        if tracks_for_album.empty:
            # No song for track something is wrong
            # print('TOSS: ' + str(cddb_row['id']) + ' ---- ' + artist_name)
            num_not_repairable += 1
            continue
        
        for _, track_row in tracks_for_album.iterrows():
            song_name = track_row['Name']
            fix = get_song_release_year(song_name, artist_name)
            sleep(1)
            if fix:
                # print(f'{artist_name} ---- {song_name} ---- {fix}')
                cddb_row['year'] = fix
                num_repairs += 1
                break # we found one we are clean
        
    print(f'Done! num repairs: {num_repairs}, num not repairable: {num_not_repairable}')
# start_repair()
    